In [4]:
!zenml init
!zenml stack set default

NumExpr defaulting to 8 threads.
Found existing ZenML repository at path 
'/home/apenner/PycharmProjects/template-starter/template'.
⠋ Initializing ZenML repository at 
/home/apenner/PycharmProjects/template-starter/template.
⠋ Initializing ZenML repository at 
/home/apenner/PycharmProjects/template-starter/template.

NumExpr defaulting to 8 threads.
Active repository stack set to: 'default'.
⠙ Setting the repository active stack to 'default'...t'...


In [5]:
# Do the imports at the top

import random
from zenml import ExternalArtifact, pipeline 
from zenml.client import Client
from zenml.logger import get_logger
from uuid import UUID

import os
from typing import Optional, List

from zenml import pipeline

from steps import (
    data_loader,
    data_preprocessor,
    data_splitter,
    model_evaluator,
    model_trainer,
    inference_predict,
    inference_preprocessor
)

logger = get_logger(__name__)

client = Client()

# Run the feature engineering pipeline

In [7]:
@pipeline
def feature_engineering(
    test_size: float = 0.2,
    drop_na: Optional[bool] = None,
    normalize: Optional[bool] = None,
    drop_columns: Optional[List[str]] = None,
    target: Optional[str] = "target",
):
    """
    Feature engineering pipeline.

    This is a pipeline that loads the data, processes it and splits
    it into train and test sets.

    Args:
        test_size: Size of holdout set for training 0.0..1.0
        drop_na: If `True` NA values will be removed from dataset
        normalize: If `True` dataset will be normalized with MinMaxScaler
        drop_columns: List of columns to drop from dataset
        target: Name of target column in dataset
    """
    ### ADD YOUR OWN CODE HERE - THIS IS JUST AN EXAMPLE ###
    # Link all the steps together by calling them and passing the output
    # of one step as the input of the next step.
    raw_data = data_loader(random_state=random.randint(0, 100), target=target)
    dataset_trn, dataset_tst = data_splitter(
        dataset=raw_data,
        test_size=test_size,
    )
    dataset_trn, dataset_tst, _ = data_preprocessor(
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        drop_na=drop_na,
        normalize=normalize,
        drop_columns=drop_columns,
        target=target,
    )
    
    return dataset_trn, dataset_tst

In [8]:
pipeline_args = {}
pipeline_args["config_path"] = os.path.join("configs", "feature_engineering.yaml")
fe_p_configured = feature_engineering.with_options(**pipeline_args)

In [9]:
latest_run = fe_p_configured()

Initiating a new run for the pipeline: feature_engineering.
Reloading configuration file /home/apenner/PycharmProjects/template-starter/template/.zen/config.yaml
Reusing registered version: (version: 1).
New model version 38 was created.
Executing a new run.
Using user: alexej@zenml.io
Using stack: default
  artifact_store: default
  orchestrator: default
Step data_loader has started.
Dataset with 541 records loaded!
Step data_loader has finished in 7.423s.
Step data_splitter has started.
Step data_splitter has finished in 10.617s.
Step data_preprocessor has started.
Step data_preprocessor has finished in 15.210s.
Run feature_engineering-2023_12_06-09_28_21_087296 has finished in 36.387s.
Dashboard URL: https://1cf18d95-zenml.cloudinfra.zenml.io/workspaces/default/pipelines/52874ade-f314-45ab-b9bf-e95fb29290b8/runs/34512253-c541-4edb-8cd2-1f4d7c0984b9/dag


# Run the training Pipeline

![title](_assets/default_stack.png)

In [10]:
@pipeline
def training(
    train_dataset_id: Optional[UUID] = None,
    test_dataset_id: Optional[UUID] = None,
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
):
    """
    Model training pipeline.

    This is a pipeline that loads the data, processes it and splits
    it into train and test sets, then search for best hyperparameters,
    trains and evaluates a model.

    Args:
        test_size: Size of holdout set for training 0.0..1.0
        drop_na: If `True` NA values will be removed from dataset
        normalize: If `True` dataset will be normalized with MinMaxScaler
        drop_columns: List of columns to drop from dataset
    """
    ### ADD YOUR OWN CODE HERE - THIS IS JUST AN EXAMPLE ###
    # Link all the steps together by calling them and passing the output
    # of one step as the input of the next step.
    
    # Execute Feature Engineering Pipeline
    if train_dataset_id is None or test_dataset_id is None:
        dataset_trn, dataset_tst = feature_engineering()
    else:
        dataset_trn = ExternalArtifact(id=train_dataset_id)
        dataset_tst = ExternalArtifact(id=test_dataset_id)
    
    model = model_trainer(
        dataset_trn=dataset_trn,
    )

    model_evaluator(
        model=model,
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        min_train_accuracy=min_train_accuracy,
        min_test_accuracy=min_test_accuracy,
    )


In [11]:
pipeline_args = {}
pipeline_args["config_path"] = os.path.join("configs", "training.yaml")
fe_t_configured = training.with_options(**pipeline_args)

In [12]:
fe_t_configured()

Initiating a new run for the pipeline: training.
Reusing registered version: (version: 2).
New model version 39 was created.
Executing a new run.
Using user: alexej@zenml.io
Using stack: default
  artifact_store: default
  orchestrator: default
Step data_loader has started.
Dataset with 541 records loaded!
Step data_loader has finished in 6.700s.
Step data_splitter has started.
Step data_splitter has finished in 11.217s.
Step data_preprocessor has started.
Step data_preprocessor has finished in 15.339s.
Caching disabled explicitly for model_trainer.
Step model_trainer has started.
Training model DecisionTreeClassifier()...
Step model_trainer has finished in 7.252s.
Step model_evaluator has started.
Train accuracy=100.00%
Test accuracy=91.74%
Implicitly linking artifact output to model breast_cancer_classifier version 39.
Step model_evaluator has finished in 6.866s.
Run training-2023_12_06-09_29_15_321058 has finished in 54.052s.
Dashboard URL: https://1cf18d95-zenml.cloudinfra.zenml.io

# Switch the Stack

![title](_assets/airflow_stack.png)

In [14]:
!zenml stack set sagemaker-airflow-stack
!zenml integration install airflow aws
!pip install apache-airflow-providers-docker apache-airflow~=2.5.0
!zenml stack up

NumExpr defaulting to 8 threads.
Active repository stack set to: 'sagemaker-airflow-stack'.
⠧ Setting the repository active stack to 'sagemaker-airflow-stack'...k'...
NumExpr defaulting to 8 threads.
All required packages for integration 'airflow' are already installed.
All required packages for integration 'aws' are already installed.
You should consider upgrading via the '/home/apenner/.pyenv/versions/3.9.13/envs/demo/bin/python -m pip install --upgrade pip' command.
NumExpr defaulting to 8 threads.
Found credentials in shared credentials file: ~/.aws/credentials
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/apenner/.config/sagemaker/config.yaml
Provisioning resources for active stack 'sagemaker-airflow-stack'.
Provisioning resources for stack 'sagemaker-airflow-stack'.
Reading the config from /home/apenner/.config/zenml/airflow/f3b0bda3-5245-4134-8ad9-6

In [15]:
step_args = {}
step_args["step_operator"] = "sagemaker-eu"
model_trainer_configured = model_trainer.with_options(**step_args)

@pipeline
def training(
    train_dataset_id: Optional[UUID] = None,
    test_dataset_id: Optional[UUID] = None,
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
):
    """
    Model training pipeline.

    This is a pipeline that loads the data, processes it and splits
    it into train and test sets, then search for best hyperparameters,
    trains and evaluates a model.

    Args:
        test_size: Size of holdout set for training 0.0..1.0
        drop_na: If `True` NA values will be removed from dataset
        normalize: If `True` dataset will be normalized with MinMaxScaler
        drop_columns: List of columns to drop from dataset
    """
    ### ADD YOUR OWN CODE HERE - THIS IS JUST AN EXAMPLE ###
    # Link all the steps together by calling them and passing the output
    # of one step as the input of the next step.
    
    # Execute Feature Engineering Pipeline
    if train_dataset_id is None or test_dataset_id is None:
        dataset_trn, dataset_tst = feature_engineering()
    else:
        dataset_trn = ExternalArtifact(id=train_dataset_id)
        dataset_tst = ExternalArtifact(id=test_dataset_id)
    
    model = model_trainer_configured(
        dataset_trn=dataset_trn,
    )

    model_evaluator(
        model=model,
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        min_train_accuracy=min_train_accuracy,
        min_test_accuracy=min_test_accuracy,
    )

In [16]:
pipeline_args = {}
pipeline_args["config_path"] = os.path.join("configs", "training.yaml")
fe_t_configured = training.with_options(**pipeline_args)

In [17]:
fe_t_configured()

Initiating a new run for the pipeline: training.
Reloading configuration file /home/apenner/PycharmProjects/template-starter/template/.zen/config.yaml
Found credentials in shared credentials file: ~/.aws/credentials
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/apenner/.config/sagemaker/config.yaml
Registered new version: (version 4).
New model version 40 was created.
Building Docker image(s) for pipeline training.
Building Docker image 715803424590.dkr.ecr.eu-central-1.amazonaws.com/zenml:training-orchestrator.
- Including integration requirements: aiohttp>=3.8.1, apache-airflow~=2.4.0, aws-profile-manager, boto3<=1.26.76, evidently>0.2.6,<0.4.5, kubernetes, mlflow>=2.1.1,<=2.6.0, mlserver-mlflow>=1.3.3, mlserver>=1.3.3, s3fs>2022.3.0,<=2023.4.0, sagemaker==2.117.0, scikit-learn<1.3, slack-sdk>=3.16.1
Could not import Azure service connector: No module na

# Switch to full Sagemaker Stack

![title](_assets/sagemaker_stack.png)

In [20]:
!zenml stack set sagemaker-stack

NumExpr defaulting to 8 threads.
Active repository stack set to: 'sagemaker-stack'.
⠸ Setting the repository active stack to 'sagemaker-stack'...k'...


In [ ]:
fe_t_configured()

Initiating a new run for the pipeline: training.
Reloading configuration file /home/apenner/PycharmProjects/template-starter/template/.zen/config.yaml
Reusing registered version: (version: 4).
New model version 42 was created.
Building Docker image(s) for pipeline training.
Building Docker image 715803424590.dkr.ecr.eu-central-1.amazonaws.com/zenml:training-orchestrator.
- Including integration requirements: aws-profile-manager, boto3<=1.26.76, kubernetes, s3fs>2022.3.0,<=2023.4.0, sagemaker==2.117.0, scikit-learn<1.3
Step 1/8 : FROM zenmldocker/zenml:0.50.0-py3.9
Step 2/8 : WORKDIR /app
Step 3/8 : COPY .zenml_integration_requirements .
Step 4/8 : RUN pip install --default-timeout=60 --no-cache-dir  -r .zenml_integration_requirements
Step 5/8 : ENV ZENML_ENABLE_REPO_INIT_WARNINGS=False
Step 6/8 : ENV ZENML_CONFIG_PATH=/app/.zenconfig
Step 7/8 : COPY . .
Step 8/8 : RUN chmod -R a+rw .
Amazon ECR requires you to create a repository before you can push an image to it. ZenML is trying to p